In [ ]:
#| default_exp madlib

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox, Stack, Layout
from traitlets import HasTraits
import os

In [ ]:
#| export
class MadLibModel(object):

    def __init__(self, description, variables, template, **kwargs):
        
        self.description = description
        # two ways to save variables... 
        for key, value in variables.items(): # ... as attributes
            setattr(self, key, value)
        self.variables = variables # ...or as a dictionary, ex. { 'input_text': 'value' }  
        self.template = template # string: '{input_text}'


In [ ]:
model_dict = {'description': '',
              'variables': {
                  'input_text': 'value'
              }, 
              'template': '{input_text}' ,
              'placeholder': 'Message AI chatbot...',
             } 
madlib_model = MadLibModel(**model_dict)
madlib_model.input_text

In [ ]:
madlib_model.variables

In [ ]:
#| export
class MadLibView(HBox):
    
    def __init__(self, model, **kwargs):
        super().__init__()
        
        self.model = model
        self.variables = {}
        for i, (key, value) in enumerate(self.model.variables.items()):
            text = Text(description=self.model.description, style={'description_width': 'initial'})
            self.variables[key] = text

        self.submit_button = widgets.Button(
            value=False,
            disabled=False,
            button_style='success',
            icon='arrow-circle-right' 
        )
        self.children = list(self.variables.values()) 

In [ ]:
madlib = MadLibView(madlib_model)
madlib

In [ ]:
madlib.variables['input_text'].value = 'hiii'

In [ ]:
madlib_model.variables

## Creating Many MadLibs from a Dictionary

In [ ]:
madlibs_list = [
    {'name': 'Open Prompt',
     'description': '',
     'variables': {
         'input_text': 'value'
        }, 
        'template': '{input_text}' ,
        'placeholder': 'Message AI chatbot...',
    },
    {
        'name': 'Metaphor',
        'variables': {
            'concept': 'value'
        }, 
        'template': "I'm having trouble understanding {concept}. Explain it as a metaphor",
        'description': "I'm having trouble understanding...", 
        'placeholder': 'why the sky is blue'
    }
]

madlibs = []
for value in madlibs_list:
    model = MadLibModel(**value)
    madlibs.append(MadLibView(model))

In [ ]:
madlibs[1]

In [ ]:
class MadLibsView(Stack):

    def __init__(self):
        super().__init__()

In [ ]:
madlibs_view = MadLibsView()

In [ ]:
for value in madlibs_list:
    model = MadLibModel(**value)
    madlibs.append(MadLib(model))
    madlibs_view.children += (MadLib(model), )

In [ ]:
madlibs_view

In [ ]:
madlibs_view?

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()